In [1]:
import pandas as pd
import eclabfiles as ecf
from matplotlib import pyplot as plt
import numpy as np
import glob
from pickle_utils import read_ecf, read_emsi, optimise_emsi_start, alt_opt_emsi_start
from config import filenames, props
%matplotlib qt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
ecfile = glob.glob(filenames.data+'*.mpr')[-1]
datfiles = glob.glob(filenames.data+'*.dat')

In [4]:
ecfdf, ecf_start = read_ecf(ecfile)

In [7]:
joined_emsi = pd.DataFrame([])
for filename in datfiles:
    emsidf, emsi_start, emsi_end = read_emsi(filename)
    if emsi_end < ecf_start:
        continue
    emsidf.loc[:, 'I'] = emsidf.I * 980
    emsi_shift = optimise_emsi_start(ecfdf, ecf_start, emsidf, emsi_start)
    emsidf.loc[:, 't'] = emsidf.t + emsi_start - ecf_start - emsi_shift
    joined_emsi = pd.concat([joined_emsi, emsidf], ignore_index=True)
plt.plot(joined_emsi.t, joined_emsi.I)
plt.plot(ecfdf.t, ecfdf.I)

[1.02221584]
[-0.23911383]
[0.79736051]
[0.02045329]
[0.0235608]
[-0.02035807]
[0.40363321]
[0.32961917]


In [8]:
ecfdf.I = 100*ecfdf.I/props.el_area

In [9]:
ecfdf['emsi'] = np.interp(ecfdf.t, joined_emsi.t, joined_emsi.emsi)
ecfdf['light'] = np.interp(ecfdf.t, joined_emsi.t, joined_emsi.light)

In [10]:
ecfdf.to_pickle(filenames.data+'data.pkl')